In [227]:
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
pd.options.mode.chained_assignment = None

In [228]:
def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    if cell is np.nan:
        return 0
    if cell.isdigit():
        return int(cell)
    else:
        return 0





In [229]:

def check_error(df:pd.DataFrame,error_df:pd.DataFrame):
    """
    Функция для проверки правильности введеденных данных
    :param df: копия датафрейма с данными из файла поо
    :param error_df: датафрейм с ошибками
    :return:
    """
    # конвертируем в инт
    df = df.applymap(check_data)


In [230]:
# создаем словарь верхнего уровня для каждого поо
high_level_dct = {}
# создаем датафрейм для регистрации ошибок
error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])

primal_df = pd.read_excel('data/poo/Брит - 2022.xlsx',skiprows=7,dtype=str)
primal_df.columns = list(map(str, primal_df.columns))
name_file = 'БРИТ'
# Заполняем пока пропуски в 15 ячейке для каждой специальности
primal_df['06'] = primal_df['06'].fillna('15 ячейка')
# Проводим проверку на корректность данных, отправляем копию датафрейма
# check_error(df.iloc[:,6:32], error_df)


In [231]:
df = primal_df.iloc[:,6:32]
df = df.applymap(check_data)

In [232]:
# получаем количество датафреймов
quantity = df.shape[0] // 15
# счетчик для обработанных датафреймов
border = 0

In [233]:
for i in range(1,quantity+1):
    temp_df = df.iloc[border:border +15,:]
    # создаем список колонок для базовой проверки 
    all_sum_cols = list(temp_df)
    # удаляем колонки 07, 09, 10
    all_sum_cols.remove('07')
    all_sum_cols.remove('09')
    all_sum_cols.remove('10')
    # получаем сумму колонок 08, 11:32
    temp_df['Сумма Основная проверка'] = temp_df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    temp_df['Результат основная проверка'] = temp_df['07'] == temp_df['Сумма Основная проверка']
    # заменяем булевые значения на понятные
    temp_df['Результат основная проверка'] = temp_df['Результат основная проверка'].apply(lambda x: 'Правильно' if x else 'Неправильно' )
    # получаем датафрейм с ошибками и извлекаем индекс
    first_error_temp_df=temp_df[temp_df['Результат основная проверка'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла','Номер строки с ошибкой','Описание ошибки',])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = first_error_temp_df['index'].tolist() # делаем список
    finish_lst_index = list(map(lambda x:x +9,raw_lst_index))
    temp_error_df['Номер строки с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = 'Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по гр.32)'
    # соединяем с главным
    error_df = pd.concat([error_df,temp_error_df],axis=0,ignore_index=True)
    # прибавляем border
    border += 15


In [234]:
error_df

,Название файла,Номер строки с ошибкой,Описание ошибки
0,БРИТ,10,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
1,БРИТ,14,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
2,БРИТ,255,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
3,БРИТ,258,Сумма гр. 07 НЕ РАВНА гр.08 + сумма(с гр.11 по...
